In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('savant_data.csv')

In [3]:
data['velocity'].mean()

np.float64(87.4898731408574)

In [4]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.07985149726776056)

In [5]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
538     36.2     36.195
1822    18.1     18.100
534     27.3     27.300
2119    29.6     29.604
561     36.6     36.594
...      ...        ...
744     29.3     29.300
733     30.3     30.300
2647    19.4     19.400
4149    25.6     25.598
3819    18.7     18.701

[915 rows x 2 columns]


In [18]:
#mean_whiff = data['swing_miss_percent'].mean()
#data['Stuff+'] = (data['swing_miss_percent'] / mean_whiff) * 100

# Normalizing the columns
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].max()) * 100
data['xwOBA_norm'] = (1 - (data['xwoba'] / data['xwoba'].max())) * 100 # Inverse
data['RunExp_norm'] = (1 - (data['run_exp'] / data['run_exp'].max())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()


,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,153.506925
1,25.7,0.311,1.273859,151.563868
2,39.5,0.261,1.302039,139.285099
3,26.0,0.285,1.117130,135.211261
4,28.9,0.281,1.143414,132.471600


In [19]:
# Sort the DataFrame by the 'Stuff+' column in descending order (higher Stuff+ is better)
features_sorted = data.sort_values(by='Stuff+', ascending=False)
features_sorted.to_csv('savant_data.csv', index=False)

In [20]:
'''
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse
data['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()
'''

"\ndata['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100\ndata['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse\ndata['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse\n\n# Assigning weights\nw1, w2, w3 = .3, .2, .5\n\n# Calculating Stuff+\ndata['Stuff+'] = (\n    w1 * data['SwingMiss_norm'] +\n    w2 * data['xwOBA_norm'] +\n    w3 * data['RunExp_norm']\n)\n\n# Display the first few rows of the Stuff+ calculation\ndata[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()\n"